In [1]:
import pandas as pd
import numpy as np 
import sys
import re
sys.path.insert(0, "modules")

import numpy as np
import pandas as pd

import plotly.express as px

import matplotlib.pyplot as plt   

import plotly.graph_objs as go
from plotly.subplots import make_subplots


## Materiality

In [2]:
merged_data=pd.read_csv('../data/merged_data.csv')


In [3]:
df_materiality=pd.read_csv('../data/raw/materiality.csv')


In [4]:
df_materiality = df_materiality.fillna('0')
df_materiality = df_materiality.drop('Field Name', axis=1)


In [5]:
for column in df_materiality.columns:
    try:
        df_materiality[column] = pd.to_numeric(df_materiality[column], errors='raise')
    except ValueError:
        pass 

df_materiality.head()

,Type,Issue,Score Field ID,Exploration & Production,Integrated Oils,Midstream,Refining & Marketing,Drilling & Drilling Support,Oilfield Services & Equipment,"Internet Media, Platforms and Services",...,Wind Energy Equipment,Renewable Energy Project Developers,"Technology Hardware, Equipment and Components",EMS/ODM,Semiconductors,Software and Tech Services,Integrated Electric Utilities and Power Generation,Electric Transmission & Dist,Gas Utilities,Water Utilities
0,Environmental,Air Quality,SR208,Y,Y,Y,Y,Y,Y,0,...,0,0,0,0,0.0,0,Y,0,0,0
1,Environmental,Climate Exposure,SR235,Y,Y,0,0,0,0,0,...,0,0,0,0,0.0,0,Y,Y,0,Y
2,Environmental,Ecological Impact,SR231,Y,Y,Y,Y,Y,Y,0,...,Y,Y,0,0,0.0,0,Y,0,Y,Y
3,Environmental,Energy Management,SR205,0,Y,0,0,Y,Y,Y,...,0,0,0,0,0.3,Y,0,0,0,Y
4,Environmental,Environmental Supply Chain Management,SR249,0,0,0,0,0,0,0,...,Y,0,0,0,0.0,0,0,0,0,0


## Reuslts-Cálculos Métricas

In [6]:
categories = df_materiality['Issue'].tolist()

results = pd.DataFrame(columns=['Industry', 'type', 'ISIN'] + categories)


for index, row in merged_data.iterrows():
    industry = row['Industry']
    isin = row['ISIN']

    result_row_env = [industry, 'Environmental', isin] + [0]*len(categories)
    result_row_soc = [industry, 'Social', isin] + [0]*len(categories)
    result_row_gov = [industry, 'Governance', isin] + [0]*len(categories)

    for index2, row2 in df_materiality.iterrows():
        score_id = row2['Score Field ID']
        category_type = row2['Type']
        issue = row2['Issue']
        
        if score_id in row.index:
            try:
                value = float(row[score_id])
            except ValueError:
                value = 0
            
            if category_type == 'Environmental':
                result_row_env[3 + categories.index(issue)] = value
            elif category_type == 'Social':
                result_row_soc[3 + categories.index(issue)] = value
            elif category_type == 'Governance':
                result_row_gov[3 + categories.index(issue)] = value

 
    result_row_env = pd.Series(result_row_env, index=['Industry', 'type', 'ISIN'] + categories)
    result_row_soc = pd.Series(result_row_soc, index=['Industry', 'type', 'ISIN'] + categories)
    result_row_gov = pd.Series(result_row_gov, index=['Industry', 'type', 'ISIN'] + categories)

  
    results = pd.concat([results, result_row_env.to_frame().T, result_row_soc.to_frame().T, result_row_gov.to_frame().T], ignore_index=True)


results

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Labor & Employment Practices,Marketing & Labeling,Occupational Health & Safety Management,Operational Risk Management,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,4.816013,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Semiconductors,Social,CNE100004ZD8,0,0,0,0,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0,0,0,0
2,Semiconductors,Governance,CNE100004ZD8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4.781484,1.60677,7.086886,7.334174
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Semiconductors,Social,US00108J1097,0,0,0,0,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,Automobiles,Social,CA35954F1080,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1040,Automobiles,Governance,CA35954F1080,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1041,Automobiles,Environmental,SGXZ55111462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1042,Automobiles,Social,SGXZ55111462,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [7]:

resultsv = results.copy()
for index, row in resultsv.iterrows():
    industry = row['Industry']
    
    if pd.notna(industry):
        if industry in df_materiality.columns:
            multiplier = df_materiality[industry].values
            
            columns_to_multiply = resultsv.columns[3:] 

            resultsv.loc[index, columns_to_multiply] *= multiplier
        else:
            print(f"Industry '{industry}' no está en df_materiality.")
    else:
        print("Warning: 'Industry' value is NaN for row index", index)

        

In [8]:

resultsv['ENVIRONMENTAL SCORE'] = resultsv[resultsv['type'] == 'Environmental'].iloc[:, 3:].sum(axis=1)
resultsv['SOCIAL SCORE'] = resultsv[resultsv['type'] == 'Social'].iloc[:, 3:].sum(axis=1)
resultsv['GOVERNANCE SCORE'] = resultsv[resultsv['type'] == 'Governance'].iloc[:, 3:].sum(axis=1)

grouped = resultsv.groupby('ISIN').sum()

grouped['ESG SCORE'] = (grouped['ENVIRONMENTAL SCORE'] * 0.4 + 
                        grouped['SOCIAL SCORE'] * 0.3 + 
                        grouped['GOVERNANCE SCORE'] * 0.3)

resultsv = resultsv.merge(grouped[['ESG SCORE']], on='ISIN', how='left')

In [9]:
resultsv['ENVIRONMENTAL SCORE'] = resultsv['ENVIRONMENTAL SCORE'].fillna(0)
resultsv['SOCIAL SCORE'] = resultsv['SOCIAL SCORE'].fillna(0)
resultsv['GOVERNANCE SCORE'] = resultsv['GOVERNANCE SCORE'].fillna(0)
resultsv['ESG SCORE'] = resultsv['ESG SCORE'].fillna(0)


C:\Users\Veronica\AppData\Local\Temp\ipykernel_46168\1307716482.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resultsv['ENVIRONMENTAL SCORE'] = resultsv['ENVIRONMENTAL SCORE'].fillna(0)
C:\Users\Veronica\AppData\Local\Temp\ipykernel_46168\1307716482.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resultsv['SOCIAL SCORE'] = resultsv['SOCIAL SCORE'].fillna(0)
C:\Users\Veronica\AppData\Local\Temp\ipykernel_46168\1307716482.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future vers

In [10]:
for column in resultsv.columns:
    try:
        resultsv[column] = pd.to_numeric(resultsv[column], errors='raise')
    except ValueError:
        pass 

resultsv.head()

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG SCORE
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,1.444804,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3.06801,0.00,0.000000,3.045588
1,Semiconductors,Social,CNE100004ZD8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,1.08,0.000000,3.045588
2,Semiconductors,Governance,CNE100004ZD8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.721334,0.385625,1.700853,1.173468,0.00000,0.00,4.981279,3.045588
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,2.030903
4,Semiconductors,Social,US00108J1097,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,1.08,0.000000,2.030903


## Mean Scores

In [11]:
cols_to_sum = resultsv.columns[3:-1]  # Excluye las primeras 3 columnas y 'GLOBAL SCORE'

sum_results = resultsv.groupby('Industry')[cols_to_sum].sum()

unique_isins_per_industry = resultsv.groupby('Industry')['ISIN'].nunique()

mean_results = sum_results.div(unique_isins_per_industry, axis=0)

mean_results['ESG SCORE'] = resultsv.groupby('Industry')['ESG SCORE'].mean()

In [12]:
mean_results

,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,Sustainable Product,Waste Management,Water Management,...,Product Quality Management,Social Supply Chain Management,Board Composition,Executive Compensation,Shareholder Rights,Audit,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG SCORE
Industry,,,,,,,,,,,,,,,,,,,,,
Automobiles,0.0,0.27052,0.0,0.000000,0.0,0.457351,0.0,1.050969,0.466498,0.000000,...,1.159411,0.0,1.588574,0.871988,0.904179,1.060332,2.245338,2.364194,4.425073,2.934915
Semiconductors,0.0,0.00000,0.0,0.967335,0.0,0.366032,0.0,0.123263,0.347286,1.141484,...,0.360398,0.0,1.755816,1.066333,1.585308,1.113278,2.945400,3.101457,5.520735,3.764817


## ENVIROMENTAL-SOCIAL-GOVERNANCE

In [13]:

Enviromental =results[results['type'] == 'Environmental']
Social =results[results['type'] == 'Social']
Governance =results[results['type'] == 'Governance']

In [14]:
columns_E= ['Access & Affordability',
 'Community Rights & Relations',
 'Customer Welfare',
 'Data Security & Customer Privacy',
 'Ethics & Compliance',
 'Labor & Employment Practices',
 'Marketing & Labeling',
 'Occupational Health & Safety Management',
 'Operational Risk Management',
 'Product Quality Management',
 'Social Supply Chain Management',
 'Board Composition',
 'Executive Compensation',
 'Shareholder Rights',
 'Audit']
columns_S= ['Air Quality',
 'Climate Exposure',
 'Ecological Impact',
 'Energy Management',
 'Environmental Supply Chain Management',
 'GHG Emissions Management',
 'Sustainable Finance',
 'Sustainable Product',
 'Waste Management',
 'Water Management', 'Board Composition',
 'Executive Compensation',
 'Shareholder Rights',
 'Audit']
columns_G= [
 'Air Quality',
 'Climate Exposure',
 'Ecological Impact',
 'Energy Management',
 'Environmental Supply Chain Management',
 'GHG Emissions Management',
 'Sustainable Finance',
 'Sustainable Product',
 'Waste Management',
 'Water Management',
 'Access & Affordability',
 'Community Rights & Relations',
 'Customer Welfare',
 'Data Security & Customer Privacy',
 'Ethics & Compliance',
 'Labor & Employment Practices',
 'Marketing & Labeling',
 'Occupational Health & Safety Management',
 'Operational Risk Management',
 'Product Quality Management',
 'Social Supply Chain Management'
]


Enviromental=Enviromental.drop(columns=columns_E)
Social=Social.drop(columns=columns_S)
Governance=Governance.drop(columns=columns_G)
Enviromental

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,Sustainable Product,Waste Management,Water Management
0,Semiconductors,Environmental,CNE100004ZD8,0.0,0.0,0.0,4.816013,0.0,0.0,0.0,0.0,5.008693,3.574166
3,Semiconductors,Environmental,US00108J1097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Semiconductors,Environmental,US0079031078,0.0,0.0,0.0,8.156052,0.0,5.900952,0.0,3.0,9.70292,8.547931
9,Semiconductors,Environmental,TW0003438008,0.0,0.0,0.0,0.0,0.0,0.342396,0.0,0.0,0.0,5.58239
12,Semiconductors,Environmental,CNE100003MM9,0.0,0.0,0.0,4.567261,0.0,0.342396,0.0,3.0,7.649168,4.819194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,Automobiles,Environmental,US0395872098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1032,Automobiles,Environmental,US54405Q2093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1035,Automobiles,Environmental,US3073597037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1038,Automobiles,Environmental,CA35954F1080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
environmental_scores = resultsv[resultsv['type'] == 'Environmental'][['ISIN', 'ENVIRONMENTAL SCORE']]
Enviromental = pd.merge(Enviromental, environmental_scores, on='ISIN', how='left')
social_scores = resultsv[resultsv['type'] == 'Social'][['ISIN', 'SOCIAL SCORE']]
Social = pd.merge(Social, social_scores, on='ISIN', how='left')
governance_scores = resultsv[resultsv['type'] == 'Governance'][['ISIN', 'GOVERNANCE SCORE']]
Governance = pd.merge(Governance, governance_scores, on='ISIN', how='left')


In [16]:
Enviromental['Mean Environmental'] = Enviromental.groupby('Industry')['ENVIRONMENTAL SCORE'].transform('mean')
Social['Mean Social'] = Social.groupby('Industry')['SOCIAL SCORE'].transform('mean')
Governance['Mean Governance'] = Governance.groupby('Industry')['GOVERNANCE SCORE'].transform('mean')


In [17]:
materiality=df_materiality.T
new_header = materiality.iloc[1]  # Seleccionar la tercera fila como nueva cabecera#
materiality = materiality[3:]  # Saltar las primeras tres filas y usar el resto como datos
materiality.columns = new_header


In [18]:
def get_zero_columns(row):
    zero_columns = row[(row == '0') | (row == 0)].index.tolist()
    return zero_columns
materiality['Zero Columns'] = materiality.apply(get_zero_columns, axis=1)
materiality=materiality[['Zero Columns']]
materiality=materiality.reset_index()
materiality = materiality.rename(columns={'index': 'Industry'})
materiality.head()

Issue,Industry,Zero Columns
0,Exploration & Production,"[Energy Management, Environmental Supply Chain..."
1,Integrated Oils,"[Environmental Supply Chain Management, Sustai..."
2,Midstream,"[Climate Exposure, Energy Management, Environm..."
3,Refining & Marketing,"[Climate Exposure, Energy Management, Environm..."
4,Drilling & Drilling Support,"[Climate Exposure, Environmental Supply Chain ..."


In [19]:
for index, row in Enviromental.iterrows():
    industry = row['Industry']
    
    # Buscar coincidencias en materiality basado en Industry
    mask = materiality['Industry'] == industry
    
    if mask.any():
        # Obtener las Zero Columns correspondientes
        zero_columns = materiality.loc[mask, 'Zero Columns'].iloc[0]
        
      
        for column in zero_columns:
            if column in Enviromental.columns:
                Enviromental.at[index, column] = np.nan



In [20]:
def clasificar_esg(esg_score, median_esg):
    if np.isnan(esg_score):
        return 'NaN'
    esg_score_int = int(round(esg_score))
    median_esg_int = int(round(median_esg))
    if esg_score_int > median_esg_int:
        return 'Above Median'
    elif esg_score_int == median_esg_int:
        return 'Leading'
    else:
        return 'Below Median'

# Crear las nuevas columnas de clasificación manejando NaNs
for column in Enviromental.columns[3:13]:
    median_col = Enviromental.groupby('Industry')[column].transform('median')
    classification_col = column + " Classification"
    Enviromental[classification_col] = Enviromental.apply(lambda row: clasificar_esg(row[column], median_col[row.name]), axis=1)

Enviromental

C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarnin

,Industry,type,ISIN,Air Quality,Climate Exposure,Ecological Impact,Energy Management,Environmental Supply Chain Management,GHG Emissions Management,Sustainable Finance,...,Air Quality Classification,Climate Exposure Classification,Ecological Impact Classification,Energy Management Classification,Environmental Supply Chain Management Classification,GHG Emissions Management Classification,Sustainable Finance Classification,Sustainable Product Classification,Waste Management Classification,Water Management Classification
0,Semiconductors,Environmental,CNE100004ZD8,NaN,NaN,NaN,4.816013,NaN,0.0,NaN,...,NaN,NaN,NaN,Above Median,NaN,Below Median,NaN,Leading,Above Median,Leading
1,Semiconductors,Environmental,US00108J1097,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,Below Median,NaN,Below Median,NaN,Leading,Below Median,Below Median
2,Semiconductors,Environmental,US0079031078,NaN,NaN,NaN,8.156052,NaN,5.900952,NaN,...,NaN,NaN,NaN,Above Median,NaN,Above Median,NaN,Above Median,Above Median,Above Median
3,Semiconductors,Environmental,TW0003438008,NaN,NaN,NaN,0.0,NaN,0.342396,NaN,...,NaN,NaN,NaN,Below Median,NaN,Below Median,NaN,Leading,Below Median,Above Median
4,Semiconductors,Environmental,CNE100003MM9,NaN,NaN,NaN,4.567261,NaN,0.342396,NaN,...,NaN,NaN,NaN,Above Median,NaN,Below Median,NaN,Above Median,Above Median,Above Median
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,Automobiles,Environmental,US0395872098,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,Below Median,NaN,NaN,NaN,Leading,NaN,Leading,Below Median,NaN
344,Automobiles,Environmental,US54405Q2093,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,Below Median,NaN,NaN,NaN,Leading,NaN,Leading,Below Median,NaN
345,Automobiles,Environmental,US3073597037,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,Below Median,NaN,NaN,NaN,Leading,NaN,Leading,Below Median,NaN
346,Automobiles,Environmental,CA35954F1080,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,Below Median,NaN,NaN,NaN,Leading,NaN,Leading,Below Median,NaN


In [21]:
for index, row in Social.iterrows():
    industry = row['Industry']
    
    # Buscar coincidencias en materiality basado en Industry
    mask = materiality['Industry'] == industry
    
    if mask.any():
        # Obtener las Zero Columns correspondientes
        zero_columns = materiality.loc[mask, 'Zero Columns'].iloc[0]
        
      
        for column in zero_columns:
            if column in Social.columns:
                Social.at[index, column] = np.nan


In [22]:
for column in Social.columns[3:14]:
    median_col = Social.groupby('Industry')[column].transform('median')
    classification_col = column + " Classification"
    Social[classification_col] = Social.apply(lambda row: clasificar_esg(row[column], median_col[row.name]), axis=1)

Social

C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Veronica\anaconda3\envs\pm2\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarnin

,Industry,type,ISIN,Access & Affordability,Community Rights & Relations,Customer Welfare,Data Security & Customer Privacy,Ethics & Compliance,Labor & Employment Practices,Marketing & Labeling,...,Community Rights & Relations Classification,Customer Welfare Classification,Data Security & Customer Privacy Classification,Ethics & Compliance Classification,Labor & Employment Practices Classification,Marketing & Labeling Classification,Occupational Health & Safety Management Classification,Operational Risk Management Classification,Product Quality Management Classification,Social Supply Chain Management Classification
0,Semiconductors,Social,CNE100004ZD8,NaN,0.0,NaN,NaN,0.0,0.0,NaN,...,Below Median,NaN,NaN,Below Median,Leading,NaN,Leading,NaN,Leading,NaN
1,Semiconductors,Social,US00108J1097,NaN,0.0,NaN,NaN,0.0,0.0,NaN,...,Below Median,NaN,NaN,Below Median,Leading,NaN,Leading,NaN,Leading,NaN
2,Semiconductors,Social,US0079031078,NaN,10.0,NaN,NaN,7.94949,6.14895,NaN,...,Leading,NaN,NaN,Above Median,Above Median,NaN,Above Median,NaN,Leading,NaN
3,Semiconductors,Social,TW0003438008,NaN,10.0,NaN,NaN,1.097494,0.0,NaN,...,Leading,NaN,NaN,Below Median,Leading,NaN,Leading,NaN,Above Median,NaN
4,Semiconductors,Social,CNE100003MM9,NaN,10.0,NaN,NaN,10.0,5.361741,NaN,...,Leading,NaN,NaN,Above Median,Above Median,NaN,Leading,NaN,Leading,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,Automobiles,Social,US0395872098,NaN,0.0,NaN,NaN,1.097494,0.0,NaN,...,Leading,NaN,NaN,Below Median,Leading,NaN,NaN,NaN,Below Median,NaN
344,Automobiles,Social,US54405Q2093,NaN,0.0,NaN,NaN,0.0,0.0,NaN,...,Leading,NaN,NaN,Below Median,Leading,NaN,NaN,NaN,Below Median,NaN
345,Automobiles,Social,US3073597037,NaN,0.0,NaN,NaN,0.0,0.0,NaN,...,Leading,NaN,NaN,Below Median,Leading,NaN,NaN,NaN,Below Median,NaN
346,Automobiles,Social,CA35954F1080,NaN,0.0,NaN,NaN,0.0,0.0,NaN,...,Leading,NaN,NaN,Below Median,Leading,NaN,NaN,NaN,Below Median,NaN


In [23]:
for column in Governance.columns[3:7]:
    median_col = Governance.groupby('Industry')[column].transform('median')
    classification_col = column + " Classification"
    Governance[classification_col] = Governance.apply(lambda row: clasificar_esg(row[column], median_col[row.name]), axis=1)

Governance

,Industry,type,ISIN,Board Composition,Executive Compensation,Shareholder Rights,Audit,GOVERNANCE SCORE,Mean Governance,Board Composition Classification,Executive Compensation Classification,Shareholder Rights Classification,Audit Classification
0,Semiconductors,Governance,CNE100004ZD8,4.781484,1.60677,7.086886,7.334174,4.981279,5.520735,Leading,Below Median,Leading,Leading
1,Semiconductors,Governance,US00108J1097,6.737669,5.417961,2.720348,8.193265,5.689677,5.520735,Above Median,Above Median,Below Median,Above Median
2,Semiconductors,Governance,US0079031078,7.321754,7.927024,7.079517,8.231827,7.554493,5.520735,Above Median,Above Median,Leading,Above Median
3,Semiconductors,Governance,TW0003438008,5.590107,3.947556,7.377765,7.552586,5.938929,5.520735,Above Median,Leading,Leading,Above Median
4,Semiconductors,Governance,CNE100003MM9,2.506994,3.522681,7.086886,6.040477,4.415290,5.520735,Below Median,Leading,Leading,Below Median
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,Automobiles,Governance,US0395872098,6.407086,5.233431,8.052547,9.220806,6.970515,4.425073,Above Median,Above Median,Above Median,Above Median
344,Automobiles,Governance,US54405Q2093,6.624016,7.7009,2.452065,6.894699,5.924509,4.425073,Above Median,Above Median,Below Median,Leading
345,Automobiles,Governance,US3073597037,0.0,0.0,0.0,0.0,0.000000,4.425073,Below Median,Below Median,Below Median,Below Median
346,Automobiles,Governance,CA35954F1080,0.0,0.0,0.0,0.0,0.000000,4.425073,Below Median,Below Median,Below Median,Below Median


In [24]:

Enviromental.to_csv('../data/enviromental.csv', index=False)
Social.to_csv('../data/social.csv', index=False)
Governance.to_csv('../data/governance.csv', index=False)

## ESG

In [25]:
ESG = resultsv[['ISIN','Industry', 'ESG SCORE']].drop_duplicates().reset_index(drop=True)


In [26]:
score1 = Enviromental[['ISIN', 'ENVIRONMENTAL SCORE']]
score2 = Social[['ISIN', 'SOCIAL SCORE']]
score3 = Governance[['ISIN', 'GOVERNANCE SCORE']]

ESG = pd.merge(ESG, score1, on='ISIN', how='left')
ESG = pd.merge(ESG, score2, on='ISIN', how='left')
ESG = pd.merge(ESG, score3, on='ISIN', how='left')

# Definir los bins y las etiquetas correspondientes
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
labels = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-7', '7-8', '8-9', '9-10']

# Crear la nueva columna 'ESG Score Range' en el DataFrame ESG
ESG['ESG Score Range'] = pd.cut(ESG['ESG SCORE'], bins=bins, labels=labels, right=False)

ESG['Mean ESG'] = ESG.groupby('Industry')['ESG SCORE'].transform('mean')



In [27]:
def clasificar_esg(esg_score, median_esg):
    esg_score_int = int(round(esg_score))
    median_esg_int = int(round(median_esg))
    if esg_score_int > median_esg_int:
        return 'Above Median'
    elif esg_score_int == median_esg_int:
        return 'Leading'
    else:
        return 'Below Median'

# Crear la nueva columna 'ESG Classification'
ESG['ESG Classification'] = ESG.apply(lambda row: clasificar_esg(row['ESG SCORE'], ESG.groupby('Industry')['ESG SCORE'].transform('median')[row.name]), axis=1)

# Crear las nuevas columnas de clasificación sin crear columnas intermedias para las medianas
ESG['E Classification'] = ESG.apply(lambda row: clasificar_esg(row['ENVIRONMENTAL SCORE'], ESG.groupby('Industry')['ENVIRONMENTAL SCORE'].transform('median')[row.name]), axis=1)
ESG['S Classification'] = ESG.apply(lambda row: clasificar_esg(row['SOCIAL SCORE'], ESG.groupby('Industry')['SOCIAL SCORE'].transform('median')[row.name]), axis=1)
ESG['G Classification'] = ESG.apply(lambda row: clasificar_esg(row['GOVERNANCE SCORE'], ESG.groupby('Industry')['GOVERNANCE SCORE'].transform('median')[row.name]), axis=1)

ESG.head()

,ISIN,Industry,ESG SCORE,ENVIRONMENTAL SCORE,SOCIAL SCORE,GOVERNANCE SCORE,ESG Score Range,Mean ESG,ESG Classification,E Classification,S Classification,G Classification
0,CNE100004ZD8,Semiconductors,3.045588,3.068010,1.080000,4.981279,3-4,3.764817,Below Median,Leading,Below Median,Leading
1,US00108J1097,Semiconductors,2.030903,0.000000,1.080000,5.689677,2-3,3.764817,Below Median,Below Median,Below Median,Above Median
2,US0079031078,Semiconductors,6.960441,7.325640,5.879457,7.554493,6-7,3.764817,Above Median,Above Median,Above Median,Above Median
3,TW0003438008,Semiconductors,3.179927,1.719228,2.368524,5.938929,3-4,3.764817,Below Median,Below Median,Below Median,Above Median
4,CNE100003MM9,Semiconductors,4.238693,4.181856,4.137878,4.415290,4-5,3.764817,Leading,Above Median,Above Median,Below Median


In [28]:
ESG.to_csv('../data/ESG_score.csv', index=False)

## esg percentage

In [29]:
esg_percentage = ESG.groupby(['Industry', 'ESG Score Range']).size().reset_index(name='Count')
total_count = ESG.groupby('Industry').size().reset_index(name='Total')
esg_percentage = esg_percentage.merge(total_count, on='Industry')
esg_percentage['Percentage'] = (esg_percentage['Count'] / esg_percentage['Total']) * 100
esg_percentage


C:\Users\Veronica\AppData\Local\Temp\ipykernel_46168\3098163376.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  esg_percentage = ESG.groupby(['Industry', 'ESG Score Range']).size().reset_index(name='Count')


,Industry,ESG Score Range,Count,Total,Percentage
0,Automobiles,0-1,3,17,17.647059
1,Automobiles,1-2,2,17,11.764706
2,Automobiles,2-3,3,17,17.647059
3,Automobiles,3-4,7,17,41.176471
4,Automobiles,4-5,0,17,0.000000
5,Automobiles,5-6,1,17,5.882353
6,Automobiles,6-7,1,17,5.882353
7,Automobiles,7-8,0,17,0.000000
8,Automobiles,8-9,0,17,0.000000
9,Automobiles,9-10,0,17,0.000000


In [30]:
esg_percentage.to_csv('../data/esg_percentage.csv', index=False)